9/14/2016

Question: how to read ARFF in Python?

In [3]:
#http://www.tecmint.com/how-to-open-extract-and-create-rar-files-in-linux/
!unrar e ./data/Chronic_Kidney_Disease.rar


UNRAR 5.30 beta 2 freeware      Copyright (c) 1993-2015 Alexander Roshal


Extracting from ./Chronic_Kidney_Disease.rar

Extracting  chronic_kidney_disease.arff                                   42%  OK 
Extracting  chronic_kidney_disease.info.txt                               49%  OK 
Extracting  chronic_kidney_disease_full.arff                              99%  OK 
All OK


Attempt 1:
* Read arff file directly with built in packages.

In [18]:
from scipy.io import arff
fn = './chronic_kidney_disease.arff'
x = arff.loadarff(fn)

ValueError:  value not in ('yes', 'no')

Note: reading with `readarff` fails

Attempt 2:
* Inspect "chronic_kidney_disease.arff" for error
* Found ",," on line 399
* Identified "?" as the missing value
* Copied file to "chronic_kidney_disease_edit.arff" and replaced ",," with ",?,"

In [26]:
fn = './chronic_kidney_disease_edit.arff'
x = arff.loadarff(fn)

ValueError: no value not in ('good', 'poor')

In [21]:
%debug

> /home/phil/anaconda/lib/python3.5/site-packages/scipy/io/arff/arffread.py(391)safe_nominal()
    389         return svalue
    390     else:
--> 391         raise ValueError("%s value not in %s" % (str(svalue), str(pvalue)))
    392 
    393 

ipdb> svalue
'no'
ipdb> q


Attempt 3: destroy arff header

* Copied "chronic_kidney_disease.arff" to "chronic_kidney_disease_edit_2.arff"
* Removed arff lines

In [22]:
fn = './chronic_kidney_disease_edit_2.arff'
x = pd.read_csv(fn)

CParserError: Error tokenizing data. C error: Expected 25 fields in line 70, saw 26


Attempt 4: Fix hanging "," on lines 70, 73
* Note: the problem on line 370 wasn't a missing "?", but rather an extra ","
* Removed that ","

In [53]:
fn = './chronic_kidney_disease_edit_2.arff'
df = pd.read_csv(fn, header=None)

Create header
* Grabbed header text from "chronic_kidney_disease.arff"
* Did column select in VIM to remove unwanted text on left side. 
* Did string processing in Python

In [42]:
s = """'age' numeric
'bp'  numeric
'sg' {1.005,1.010,1.015,1.020,1.025}
'al' {0,1,2,3,4,5}  
'su' {0,1,2,3,4,5}  
'rbc' {normal,abnormal}
'pc' {normal,abnormal} 
'pcc' {present,notpresent}
'ba' {present,notpresent}
'bgr'  numeric
'bu' numeric
'sc' numeric
'sod' numeric
'pot' numeric
'hemo' numeric
'pcv' numeric
'wbcc' numeric
'rbcc' numeric
'htn' {yes,no}
'dm' {yes,no}
'cad' {yes,no}
'appet' {good,poor}
'pe' {yes,no} 
'ane' {yes,no}
'class' {ckd,notckd}"""

In [49]:
header = []
for x in s.split('\n'):
    header.append(x.split()[0])

#Example with list comprehensions
header = [x.split()[0].replace("'","") for x in s.split('\n')]

In [58]:
df.columns = header

In [60]:
df.head()

age  bp     sg al su     rbc        pc         pcc          ba  bgr  ...   \
0  48  80  1.020  1  0       ?    normal  notpresent  notpresent  121  ...    
1   7  50  1.020  4  0       ?    normal  notpresent  notpresent    ?  ...    
2  62  80  1.010  2  3  normal    normal  notpresent  notpresent  423  ...    
3  48  70  1.005  4  0  normal  abnormal     present  notpresent  117  ...    
4  51  80  1.010  2  0  normal    normal  notpresent  notpresent  106  ...    

  pcv  wbcc rbcc  htn   dm cad appet   pe  ane class  
0  44  7800  5.2  yes  yes  no  good   no   no   ckd  
1  38  6000    ?   no   no  no  good   no   no   ckd  
2  31  7500    ?   no  yes  no  poor   no  yes   ckd  
3  32  6700  3.9  yes   no  no  poor  yes  yes   ckd  
4  35  7300  4.6   no   no  no  good   no   no   ckd  

[5 rows x 25 columns]